Connect google drive



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For clearing outputs import clear and for displaying images import Image

In [ ]:
from google.colab.output import clear
from IPython.display import Image

# Data Processing

## Downloading and Extracting

Downloading the dataset

In [ ]:
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-train-data.zip
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-test-data.zip
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-Train-Annotations-XML.zip
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-Test-Annotations-XML.zip
clear()

--2023-05-01 16:49:59--  http://detrac-db.rit.albany.edu/Data/DETRAC-train-data.zip
Resolving detrac-db.rit.albany.edu (detrac-db.rit.albany.edu)... 169.226.48.9
Connecting to detrac-db.rit.albany.edu (detrac-db.rit.albany.edu)|169.226.48.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://detrac-db.rit.albany.edu:443/Data/DETRAC-train-data.zip [following]
--2023-05-01 16:49:59--  https://detrac-db.rit.albany.edu/Data/DETRAC-train-data.zip
Connecting to detrac-db.rit.albany.edu (detrac-db.rit.albany.edu)|169.226.48.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5614357445 (5.2G) [application/zip]
Saving to: ‘DETRAC-train-data.zip’

DETRAC-train-data.z 100%[===================>]   5.23G   132MB/s    in 30s     

2023-05-01 16:50:29 (179 MB/s) - ‘DETRAC-train-data.zip’ saved [5614357445/5614357445]

--2023-05-01 16:50:29--  http://detrac-db.rit.albany.edu/Data/DETRAC-test-data.zip
Resolving detrac-db.rit.albany.edu (detrac-

Create folder for extracting the dataset

In [ ]:
!mkdir data/

Unzip downloaded files

In [ ]:
!unzip -q DETRAC-train-data.zip -d data/
!unzip -q DETRAC-test-data.zip -d data/
!unzip -q DETRAC-Train-Annotations-XML.zip -d data/
!unzip -q DETRAC-Test-Annotations-XML.zip -d data/

Remove zip files after extacting zip is done

In [ ]:
!rm -r *.zip

## Processing Data

Copy the file to process our data so that it can have a structure that is required by YOLOv8

In [ ]:
!cp /content/drive/MyDrive/DETRAC/process.py ./data

Process the Files so that it will be compatible with the YOLOv8

In [ ]:
%cd /content/data

/content/data


In [ ]:
!python process.py

100% 60/60 [01:55<00:00,  1.92s/it]
YOLO training set image has been generated.
100% 40/40 [01:15<00:00,  1.88s/it]
YOLO validation set image has been generated.
100% 60/60 [1:14:04<00:00, 74.08s/it]
YOLO training set label has been generated.
100% 40/40 [35:37<00:00, 53.45s/it]
YOLO validation set label has been generated.
Mission accomplished.


Now we have the training and validation split but we need the test split as well. Since the testing dataset from DETRAC is labeled we will be using half of it for validation and half for testing

In [ ]:
destImgPath = "test/images"
destlabelPath = "test/labels"

In [ ]:
!mkdir test
!mkdir $destImgPath $destlabelPath

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm
import shutil

imagesPath = "valid/images/"
labelsPath = "valid/labels/"

images = os.listdir(imagesPath)
labels = os.listdir(labelsPath)

numImages = len(images)

# half for test and half for validation
halfImages = numImages // 2
count = 0

for i in tqdm(images):
  count += 1
  if count < halfImages:
    continue
  name = Path(i).stem
  textFile = name + ".txt"
  if name + ".txt" in labels:
    shutil.move(imagesPath + i, destImgPath)
    shutil.move(labelsPath + textFile, destlabelPath)
  else:
    shutil.move(imagesPath + i, destImgPath)

100%|██████████| 56340/56340 [00:22<00:00, 2465.75it/s]


Compress the processed files

In [ ]:
!zip -qr DETRAC.zip train/ valid/ test/

Copy the files to google drive

In [ ]:
dest = '/content/drive/MyDrive/Dataset Final'

In [ ]:
! cp -r '/content/data/DETRAC.zip' $dest

Remove unnecessary files

In [ ]:
!rm -rf DETRAC.zip \
DETRAC-Test-Annotations-XML/ \
DETRAC-Train-Annotations-XML/ \
Insight-MVT_Annotation_Test/ \
Insight-MVT_Annotation_Train/ \
process.py 